<a href="https://colab.research.google.com/github/Nungdoo/deep-learning-study/blob/main/exercise04_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비하기

데이터 세팅

In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

x = cancer.data
y = cancer.target

훈련 세트와 테스트 세트로 나누기

train_test_split() 함수
- 기본적으로 훈련 세트 75%, 테스트 세트 25%의 비율로 나누어 줌
- 매개변수로 x(data), y(target)를 넘겨줌
- stratify=y : 훈련 데이터를 나눌 때, 양성/음성 클래스의 비율 유지
- test_size=0.2 : 디폴트 비율(75:25)가 아닌 80:20으로 훈련 데이터 나누기
- random_state=42 : 실험 결과를 똑같이 재현하기 위해 난수 42로 지정, 실전에서는 입력하지 않고 데이터를 무작위로 섞음

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

# 나눠진 데이터 비율 확인
# 455:114 로 4:1로 잘 나뉨
print(x_train.shape, x_test.shape)

# 타깃에 속한 클래스의 개수 확인
np.unique(y_train, return_counts=True)

(455, 30) (114, 30)


(array([0, 1]), array([170, 285]))

# 로지스틱 회귀 구현하기

In [4]:
class LogisticNeuron:

  def __init__(self):
    self.w = None
    self.b = None

  def forpass(self, x):
    z = np.sum(x * self.w) + self.b   # 직선 방정식을 계산합니다.
    return z

  def backprop(self, x, err):
    w_grad = x * err  # 가중치에 대한 그레이디언트를 계산합니다.
    b_grad = 1 * err  # 절편에 대한 그레이디언트를 계산합니다.
    return w_grad, b_grad

  def activation(self, z):
    a = 1 / (1 + np.exp(-z))  # 시그모이드 계산
    return a

  def fit(self, x, y, epochs=100):
    self.w = np.ones(x.shape[1])
    self.b = 0
    for i in range(epochs):
      for x_i, y_i in zip(x, y):
        z = self.forpass(x_i)   # 정방향 계산
        a = self.activation(z)  # 활성화 함수 적용
        err = -(y_i - a)        # 오차 계산
        w_grad, b_grad = self.backprop(x_i, err)  # 역방향 계산
        self.w -= w_grad
        self.b -= b_grad

  def predict(self, x):
    z = [self.forpass(x_i) for x_i in x]
    a = self.activation(np.array(z))
    return a > 0.5


__init__()
- 이전에는 init 함수에서 가중치와 절편을 초기화 했었음
- 입력 데이터의 특성이 많아, 가중치를 미리 초기화 하지 않고, 나중에 입력 데이터를 보고 특성 개수에 맞게 결정

forpass()
- 가중치와 입력 특성의 곱을 모두 더하기 위해 np.sum() 함수 사용
- x와 w는 1차원 배열
- w1x2 + w2x2 + w3x3 ...

activation()
- 활성화 함수

fit()
- 훈련하는 함수
- np.ones() : 입력된 매개변수와 동일한 크기의 배열을 만들고 값을 모두 1로 채움
- 초기화 한 self.w.shape : (30,)

predict()
- 리스트 내포 : 대괄호 안에 for문 삽입하여 새 리스트 생성
- 선형 함수, 활성화 함수, 임계 함수 순으로 진행해 예측값을 구함

# 로지스틱 회귀 모델 훈련시키기

모델 훈련하기

In [8]:
neuron = LogisticNeuron()
neuron.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp


모델의 정확도 평가하기
- True/False로 채워진 배열과 0/1로 채워진 배열(y_test)의 각 요소를 비교하여 평균 계산

In [9]:
np.mean(neuron.predict(x_test) == y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp


0.8245614035087719